<a href="https://colab.research.google.com/github/buthaina279/LSTM_MIMIC-III/blob/main/prepare_vitals_labs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#for google drive
DATA_FILEPATH = "/content/drive/MyDrive/final_project/all_hourly_data.h5"

In [ ]:
import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss

Read the file of the vital signs and 
laboratory tests

In [ ]:
%%time
data_full_lvl2 = pd.read_hdf(DATA_FILEPATH, 'vitals_labs')

CPU times: user 8.12 s, sys: 6.09 s, total: 14.2 s
Wall time: 49.7 s


Show a sample

In [ ]:
data_full_lvl2.head()

LEVEL2                                 alanine aminotransferase             \
Aggregation Function                                      count  mean  std   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                             2.0  25.0  0.0   
                              1                             0.0   NaN  NaN   
                              2                             0.0   NaN  NaN   
                              3                             0.0   NaN  NaN   
                              4                             0.0   NaN  NaN   

LEVEL2                                 albumin           albumin ascites       \
Aggregation Function                     count mean  std           count mean   
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0            2.0  1.8  0.0             0.0  NaN   
                              1            0.0  NaN  NaN             0.0  NaN   
                              2            0.0  NaN  NaN             0.0  NaN   
                              3            0.0  NaN  NaN             0.0  NaN   
                              4            0.0  NaN  NaN             0.0  NaN   

LEVEL2                                     albumin pleural  ...  \
Aggregation Function                   std           count  ...   
subject_id hadm_id icustay_id hours_in                      ...   
3          145834  211552     0        NaN             0.0  ...   
                              1        NaN             0.0  ...   
                              2        NaN             0.0  ...   
                              3        NaN             0.0  ...   
                              4        NaN             0.0  ...   

LEVEL2                                 white blood cell count  \
Aggregation Function                                      std   
subject_id hadm_id icustay_id hours_in                          
3          145834  211552     0                      4.012837   
                              1                           NaN   
                              2                           NaN   
                              3                           NaN   
                              4                           NaN   

LEVEL2                                 white blood cell count urine           \
Aggregation Function                                          count mean std   
subject_id hadm_id icustay_id hours_in                                         
3          145834  211552     0                                 0.0  NaN NaN   
                              1                                 0.0  NaN NaN   
                              2                                 0.0  NaN NaN   
                              3                                 0.0  NaN NaN   
                              4                                 0.0  NaN NaN   

LEVEL2                                    ph                 ph urine           
Aggregation Function                   count  mean       std    count mean std  
subject_id hadm_id icustay_id hours_in                                          
3          145834  211552     0          9.0  7.40  0.147733      1.0  5.0 NaN  
                              1          0.0   NaN       NaN      0.0  NaN NaN  
                              2          3.0  7.26  0.000000      0.0  NaN NaN  
                              3          0.0   NaN       NaN      0.0  NaN NaN  
                              4          0.0   NaN       NaN      0.0  NaN NaN  

[5 rows x 312 columns]

Check the number of levels in the hierarchical index

In [ ]:
nblevels = data_full_lvl2.index.nlevels
print("the number of levels are", nblevels )

the number of levels are 4


Flatten multiindex columns

In [ ]:
#1
data_full_lvl2.columns = ['_'.join(col).strip() for col in data_full_lvl2.columns.values]

In [ ]:
data_full_lvl2.head()

alanine aminotransferase_count  \
subject_id hadm_id icustay_id hours_in                                   
3          145834  211552     0                                    2.0   
                              1                                    0.0   
                              2                                    0.0   
                              3                                    0.0   
                              4                                    0.0   

                                        alanine aminotransferase_mean  \
subject_id hadm_id icustay_id hours_in                                  
3          145834  211552     0                                  25.0   
                              1                                   NaN   
                              2                                   NaN   
                              3                                   NaN   
                              4                                   NaN   

                                        alanine aminotransferase_std  \
subject_id hadm_id icustay_id hours_in                                 
3          145834  211552     0                                  0.0   
                              1                                  NaN   
                              2                                  NaN   
                              3                                  NaN   
                              4                                  NaN   

                                        albumin_count  albumin_mean  \
subject_id hadm_id icustay_id hours_in                                
3          145834  211552     0                   2.0           1.8   
                              1                   0.0           NaN   
                              2                   0.0           NaN   
                              3                   0.0           NaN   
                              4                   0.0           NaN   

                                        albumin_std  albumin ascites_count  \
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0                 0.0                    0.0   
                              1                 NaN                    0.0   
                              2                 NaN                    0.0   
                              3                 NaN                    0.0   
                              4                 NaN                    0.0   

                                        albumin ascites_mean  \
subject_id hadm_id icustay_id hours_in                         
3          145834  211552     0                          NaN   
                              1                          NaN   
                              2                          NaN   
                              3                          NaN   
                              4                          NaN   

                                        albumin ascites_std  \
subject_id hadm_id icustay_id hours_in                        
3          145834  211552     0                         NaN   
                              1                         NaN   
                              2                         NaN   
                              3                         NaN   
                              4                         NaN   

                                        albumin pleural_count  ...  \
subject_id hadm_id icustay_id hours_in                         ...   
3          145834  211552     0                           0.0  ...   
                              1                           0.0  ...   
                              2                           0.0  ...   
                              3                           0.0  ...   
                              4                           0.0  ...   

                                        white blood cell count_std  \
subject

Reset index of the DataFrame

In [ ]:
#2
df = data_full_lvl2.reset_index()
df.head()

,subject_id,hadm_id,icustay_id,hours_in,alanine aminotransferase_count,alanine aminotransferase_mean,alanine aminotransferase_std,albumin_count,albumin_mean,albumin_std,...,white blood cell count_std,white blood cell count urine_count,white blood cell count urine_mean,white blood cell count urine_std,ph_count,ph_mean,ph_std,ph urine_count,ph urine_mean,ph urine_std
0,3,145834,211552,0,2.0,25.0,0.0,2.0,1.8,0.0,...,4.012837,0.0,NaN,NaN,9.0,7.40,0.147733,1.0,5.0,NaN
1,3,145834,211552,1,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
2,3,145834,211552,2,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,0.0,NaN,NaN,3.0,7.26,0.000000,0.0,NaN,NaN
3,3,145834,211552,3,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
4,3,145834,211552,4,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN


Check the DataFrame shape

In [ ]:
df.shape

(2200954, 316)

Check how many icu stay in this dataset

In [ ]:
IDs = list(set(df['icustay_id'].tolist())) # get all the time series ID's
num_samples = len(IDs)
print(num_samples)

34472


Remove count columns

In [ ]:
df_no_count = df.loc[:, ~df.columns.str.endswith('_count')]
df_no_count.shape

(2200954, 212)

Extract the first 24 hours 

In [ ]:
rslt_df = df_no_count[df_no_count['hours_in'] < 24]

In [ ]:
rslt_df.shape

(808539, 212)

In [ ]:
rslt_df.head()

,subject_id,hadm_id,icustay_id,hours_in,alanine aminotransferase_mean,alanine aminotransferase_std,albumin_mean,albumin_std,albumin ascites_mean,albumin ascites_std,...,weight_mean,weight_std,white blood cell count_mean,white blood cell count_std,white blood cell count urine_mean,white blood cell count urine_std,ph_mean,ph_std,ph urine_mean,ph urine_std
0,3,145834,211552,0,25.0,0.0,1.8,0.0,NaN,NaN,...,NaN,NaN,14.842857,4.012837,NaN,NaN,7.40,0.147733,5.0,NaN
1,3,145834,211552,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,145834,211552,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.26,0.000000,NaN,NaN
3,3,145834,211552,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,145834,211552,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check how many hours for each ICU stay

In [ ]:
count_hours_icu = rslt_df.groupby('icustay_id').count()
count_hours_icu.head()

,subject_id,hadm_id,hours_in,alanine aminotransferase_mean,alanine aminotransferase_std,albumin_mean,albumin_std,albumin ascites_mean,albumin ascites_std,albumin pleural_mean,...,weight_mean,weight_std,white blood cell count_mean,white blood cell count_std,white blood cell count urine_mean,white blood cell count urine_std,ph_mean,ph_std,ph urine_mean,ph urine_std
icustay_id,,,,,,,,,,,,,,,,,,,,,
200003,24,24,24,3,2,2,1,0,0,0,...,0,0,3,2,0,0,4,4,1,0
200007,24,24,24,0,0,1,1,0,0,0,...,1,0,2,1,0,0,0,0,0,0
200009,24,24,24,0,0,0,0,0,0,0,...,1,0,6,6,0,0,9,8,0,0
200014,24,24,24,1,1,2,2,0,0,0,...,0,0,2,2,0,0,6,6,0,0
200019,24,24,24,0,0,0,0,0,0,0,...,0,0,2,1,0,0,1,1,1,0


Check missing values from the original dataset

In [ ]:
def missing(df):
        # Total missing values
        val = df.isnull().sum()
        
        # Percentage of missing values
        percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        table = pd.concat([val, percent], axis=1)
        
        # Rename the columns
        table_ren_columns = table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        table_ren_columns = table_ren_columns[table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        # Return the dataframe with missing information
        return table_ren_columns

In [ ]:
missing(df)

,Missing Values,% of Total Values
lymphocytes pleural_std,2200954,100.0
albumin pleural_std,2200954,100.0
lymphocytes ascites_std,2200954,100.0
red blood cell count ascites_std,2200954,100.0
red blood cell count pleural_std,2200954,100.0
...,...,...
mean blood pressure_mean,301798,13.7
diastolic blood pressure_mean,292280,13.3
systolic blood pressure_mean,291810,13.3
respiratory rate_mean,260500,11.8


In [ ]:
rslt_df.to_csv('/content/drive/MyDrive/final_project/vitals_labs.csv') 